In [ ]:
import pandas as pd
import numpy as np 
import os

print('Import complete')


In [ ]:
# create an empty data frame
epl_df = pd.DataFrame()
epl_df.shape

# loop through the csv files in the data-csv folder
import_dir  = 'c:\premier League\data-csv'

for entry in os.scandir(import_dir):
    if (entry.path.endswith(".csv")) and (entry.name.startswith("Season-")) and entry.is_file():   
        # print('File Name : ', entry.name)
        # create a DF from the csv file        
        df = pd.read_csv(entry, index_col=False)
        # fill any NaN values with zeroes
        # df = df.fillna(0)                               
        #append the DF to the epl_df - index all rows 1,2,3.....etc
        epl_df = pd.concat([epl_df, df], ignore_index=True)

Print('DataFrame Completed')
        

In [ ]:
epl_df.shape
# clean the data up
# remove blank rows
# epl_df.dropna(inplace = True)
# replace NaN values with blank
# epl_df.fillna('', inplace = True)
# epl_df

In [ ]:
epl_df

In [ ]:
# write the epl_df dataframe to as CSV file
# epl_df.to_csv(r'data-csv\all-seasons.csv', index = False)
# write all rows for the columns : Date, HomeTeam, AwayTeam, FTHG, FTAG, FTR, HTHG, HTAG, HTR
try:
    epl_df.loc[ : , 'Date' : 'HTR' ].to_csv(r'data-csv\all-seasons.csv', index = False)
    print('File Created')
except:
    print('Something went wrong - check if file is open in Excel')

In [147]:
historical_data = pd.read_csv(r'c:\premier league\data-csv\all-seasons.csv')
historical_data

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR
0,19/08/00,Charlton,Man City,4.0,0.0,H,2.0,0.0,H
1,19/08/00,Chelsea,West Ham,4.0,2.0,H,1.0,0.0,H
2,19/08/00,Coventry,Middlesbrough,1.0,3.0,A,1.0,1.0,D
3,19/08/00,Derby,Southampton,2.0,2.0,D,1.0,2.0,A
4,19/08/00,Leeds,Everton,2.0,0.0,H,2.0,0.0,H
...,...,...,...,...,...,...,...,...,...
10696,14/05/2000,Sheffield Weds,Leicester,4.0,0.0,H,2.0,0.0,H
10697,14/05/2000,Southampton,Wimbledon,2.0,0.0,H,0.0,0.0,D
10698,14/05/2000,Tottenham,Sunderland,3.0,1.0,H,1.0,1.0,D
10699,14/05/2000,Watford,Coventry,1.0,0.0,H,1.0,0.0,H


In [ ]:
# current season data file is Season-20-21.csv

# create an empty dictionary 
league_table = dict()

# open the csv file for the current season
current_season = open(r'data-csv\Season-20-21.csv', 'rt', encoding='UTF8')

# parse each match in the CSV file matches
for match in current_season:
    
    #trim match
    match = match.rstrip()
    
    # split the match into parts, comma as separator
    match_parts = match.split(',')
    
    # skip the header line, which has "Div" as first column
    if match_parts[0] == "Div" : continue

    # should include a check at some point to see if the Time column is included in the data file
    
    ht  = match_parts[2]
    at  = match_parts[3]
    res = match_parts[6]

    # this creates a new Key or retrieves an existing Key
    # if a new Key then the Value is 0
    # if an existing Key, the Value is incremented by the points for W/L/D
    if res == 'H':
        league_table[ht] = league_table.get(ht, 0) + 3
    elif res == 'A':
        league_table[at] = league_table.get(at, 0) + 3
    elif res == 'D':
        league_table[ht] = league_table.get(ht, 0) + 1
        league_table[at] = league_table.get(at, 0) + 1

print(league_table)

# close the file handle
current_season.close()


In [ ]:
# create list of positions
position = [*range(1, len(league_table)+1, 1)]
# print(position)

# create a DataFrame to display the current league standings
premiership = pd.DataFrame.from_dict(league_table, orient='index', columns=['Points'])

# make team a column rather than an index
premiership.reset_index(inplace=True)
premiership = premiership.rename(columns = {'index':'Team'})

# sort highest to lowest
premiership.sort_values(by=['Points'], ascending=False, inplace=True)

# add the columns to indicate league position
premiership['Position'] = position

# Make the Position column the index
premiership.set_index('Position', inplace=True)

# print the league table
premiership


In [ ]:
TeamA = 'Arsenal'
TeamB = 'Leeds'

# can use SUM as there is only a single value returned so its the sum of itself
TeamAPoints = premiership.loc[premiership.Team == TeamA, 'Points'].sum()
TeamBPoints = premiership.loc[premiership.Team == TeamB, 'Points'].sum()

# get the index for the row for each team
TeamAPos = premiership[premiership['Team'] == TeamA].index[0]
TeamBPos = premiership[premiership['Team'] == TeamB].index[0]

# calculate the number of games played between the two teams
played = epl_df.loc[((epl_df["HomeTeam"].isin([TeamA, TeamB])) & (epl_df["AwayTeam"].isin([TeamA, TeamB]))), 'Date' : 'HTR' ]

# Number of games played
games_played = played.FTR.count()

# How many draws
TeamADraws    = played.loc[ ( (played.FTR == 'D') ), 'FTR' ].count()
# how many home/away wins
TeamAHomeWins = played.loc[ ( (played.FTR == 'H') & (played.HomeTeam == TeamA) ), 'FTR' ].count()
TeamAAwayWins = played.loc[ ( (played.FTR == 'A') & (played.AwayTeam == TeamA) ), 'FTR' ].count()
# how many home/away wins
TeamBHomeWins = played.loc[ ( (played.FTR == 'H') & (played.HomeTeam == TeamB) ), 'FTR' ].count()
TeamBAwayWins = played.loc[ ( (played.FTR == 'A') & (played.AwayTeam == TeamB) ), 'FTR' ].count()

# print('Played : %d  Draws : %d'%(games_played, TeamADraws))
# print('%s \t Home : %d \t Away : %d'%(TeamA, TeamAHomeWins, TeamAAwayWins))
# print('%s \t\t Home : %d \t Away : %d'%(TeamB, TeamBHomeWins, TeamBAwayWins))

print('#===================================================================================================')
print('Team \t\t Position \t Points \t Played \t Home Wins \t Away Wins \t Draws')
print('#===================================================================================================')
print('%s \t %d \t\t %d \t\t %d \t\t %d \t\t %d \t\t %d'%(TeamA, TeamAPos, TeamAPoints, games_played, TeamAHomeWins, TeamAAwayWins, TeamADraws))
print('%s \t\t %d \t\t %d \t\t %d \t\t %d \t\t %d \t\t %d'%(TeamB, TeamBPos, TeamBPoints, games_played, TeamBHomeWins, TeamBAwayWins, TeamADraws))
print('#===================================================================================================')

played


In [ ]:
from itertools import groupby

#what is the longest winning sequence TeamA has ever had
winning_sequence = (( (historical_data.loc[ ((historical_data.HomeTeam == TeamA) & (historical_data.FTR == 'H') ) | ((historical_data.AwayTeam == TeamA) & (historical_data.FTR == 'A') ), 'FTR' ]) ))

print(winning_sequence.head(30))

substrings = [list(repeated_chars) for _, repeated_chars in groupby(winning_sequence)]

# print(substrings)

longest = 0
winning_run = None

for substr in substrings:
    if (len(substr)) > longest:
        longest = len(substr)
        winning_run = substr

print('The longest winning run was %d matches, %s'%(longest, winning_run))

print(substrings)


In [127]:

#what is the longest winning sequence TeamA has had againt TeamB
winning_sequence = (( (played.loc[ ((played.HomeTeam == TeamA) & (played.FTR == 'H') ) | ((played.AwayTeam == TeamA) & (played.FTR == 'A') ), 'FTR' ]) ))

print(winning_sequence.head(30))

substrings = [list(repeated_chars) for _, repeated_chars in groupby(winning_sequence)]

# print(substrings)

longest = 0
winning_run = None

for substr in substrings:
    if (len(substr)) > longest:
        longest = len(substr)
        winning_run = substr

print('%as longest winning run against %s was %d matches, %s'%(TeamA, TeamB, longest, winning_run))

print(substrings)


356      H
833      A
1241     A
1467     H
7832     H
7911     H
8884     A
9123     H
9280     H
9769     H
10119    H
10507    H
10650    A
Name: FTR, dtype: object
'Arsenal's longest winning run against Leeds was 5 matches, ['H', 'H', 'H', 'H', 'H']
[['H'], ['A', 'A'], ['H', 'H', 'H'], ['A'], ['H', 'H', 'H', 'H', 'H'], ['A']]


In [130]:
# need to calculate current form for both Teams, how many Goals scored, conceded etc
played.value_counts()

Date        HomeTeam  AwayTeam  FTHG  FTAG  FTR  HTHG  HTAG  HTR
01/02/1997  Leeds     Arsenal   0.0   0.0   D    0.0   0.0   D      1
01/11/03    Leeds     Arsenal   1.0   4.0   A    0.0   3.0   A      1
28/09/2002  Leeds     Arsenal   1.0   4.0   A    0.0   2.0   A      1
26/11/00    Leeds     Arsenal   1.0   0.0   H    0.0   0.0   D      1
26/10/1996  Arsenal   Leeds     3.0   0.0   H    2.0   0.0   H      1
22/11/2020  Leeds     Arsenal   0.0   0.0   D    0.0   0.0   D      1
21/08/01    Arsenal   Leeds     1.0   2.0   A    1.0   1.0   D      1
20/12/1998  Arsenal   Leeds     3.0   1.0   H    1.0   0.0   H      1
20/01/02    Leeds     Arsenal   1.0   1.0   D    1.0   1.0   D      1
16/04/2000  Leeds     Arsenal   0.0   4.0   A    0.0   1.0   A      1
16/04/04    Arsenal   Leeds     5.0   0.0   H    3.0   0.0   H      1
14/10/1995  Leeds     Arsenal   0.0   3.0   A    0.0   1.0   A      1
14/02/2021  Arsenal   Leeds     4.0   2.0   H    3.0   0.0   H      1
11/05/1999  Leeds     Ars

In [210]:
# calculate current form for each Team in the current season
# how many goals 

TeamA_FTHG = (( (current_form.loc[ ((current_form.HomeTeam == TeamA) & (current_form.FTR == 'H') ) | ((current_form.AwayTeam == TeamA) & (current_form.FTR == 'A') ), 'FTHG' ]) ))

TeamB_FTHG = (( (current_form.loc[ ((current_form.HomeTeam == TeamB) & (current_form.FTR == 'H') ) | ((current_form.AwayTeam == TeamB) & (current_form.FTR == 'A') ), 'FTHG' ]) ))

TeamA_FTAG = (( (current_form.loc[ ((current_form.AwayTeam == TeamA) & (current_form.FTR == 'A') ) | ((current_form.HomeTeam == TeamA) & (current_form.FTR == 'A') ), 'FTHG' ]) ))

TeamB_FTAG = (( (current_form.loc[ ((current_form.HomeTeam == TeamB) & (current_form.FTR == 'H') ) | ((current_form.AwayTeam == TeamB) & (current_form.FTR == 'A') ), 'FTHG' ]) ))

#print(TeamA, ' : ', TeamA_FTHG.head(10).sum())
#print(TeamB, ' : ', TeamB_FTHG.head(10).sum())

current_form = pd.read_csv(r'data-csv\Season-20-21.csv', parse_dates=["Date"], dayfirst=True)

# current_form.set_index('Date', inplace=True)

# sort highest to lowest
current_form.sort_values(by=['Date'], ascending=False, inplace=True)

# current_form["Date"].min(), current_form["Date"].max()
# current_form.sort_index(axis=0, ascending=False)

current_form


,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA
275,E0,2021-03-10,Man City,Southampton,5,2,H,3,1,H,...,2.38,-2.00,2.07,1.83,2.06,1.85,2.10,1.95,2.03,1.85
274,E0,2021-03-08,West Ham,Leeds,2,0,H,2,0,H,...,2.10,-0.25,1.95,1.98,1.94,1.98,2.00,2.00,1.96,1.94
273,E0,2021-03-08,Chelsea,Everton,2,0,H,1,0,H,...,1.77,-0.75,1.88,2.05,1.88,2.04,1.88,2.14,1.82,2.08
272,E0,2021-03-07,Tottenham,Crystal Palace,4,1,H,1,1,D,...,1.87,-1.00,2.02,1.91,2.02,1.91,2.04,2.01,1.95,1.93
271,E0,2021-03-07,Man City,Man United,0,2,A,0,1,A,...,1.96,-1.00,1.88,2.05,1.86,2.07,1.90,2.08,1.85,2.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,E0,2020-09-13,West Brom,Leicester,0,3,A,0,0,D,...,1.73,0.25,1.92,1.98,1.93,1.99,1.95,2.01,1.91,1.97
2,E0,2020-09-12,Liverpool,Leeds,4,3,H,3,2,H,...,2.62,-1.50,1.85,2.05,1.85,2.08,1.90,2.16,1.84,2.04
1,E0,2020-09-12,Crystal Palace,Southampton,1,0,H,1,0,H,...,1.70,0.25,1.78,2.13,1.79,2.17,1.85,2.18,1.79,2.12
3,E0,2020-09-12,West Ham,Newcastle,0,2,A,0,0,D,...,1.92,-0.50,2.03,1.87,2.04,1.88,2.09,1.91,2.02,1.86
